In [1]:
import sys
import os
from photutils.detection import DAOStarFinder
sys.path.append(os.path.abspath('../src'))
from data_processing import get_coords
from analysis import calculate_flux, magn_finder
import numpy as np


In [1]:
my_path = '/home/vladislav/Education/3 курс/astro_prac/photometry_of_variable_stars/data/'


In [ ]:
get_coords(my_path, 'h', 5000, 4.0)
get_coords(my_path, 'k', 3000, 3.0)
get_coords(my_path, 'j', 2000, 4.0)

In [ ]:
get_times('/home/vladislav/Education/3 курс/astro_prac/photometry_of_variable_stars/data/','h')

dates = pd.DataFrame(data)
df_h = df_h.join(dates[['jd', 'observation_time']])

df_h = df_h.dropna()
df_h['Опорная1'] = list(zip(df_h['xОпорная1'],df_h['yОпорная1']))
df_h['Опорная2'] = list(zip(df_h['xОпорная2'],df_h['yОпорная2']))
df_h['Переменная1'] = list(zip(df_h['xПеременная1'],df_h['yПеременная1']))
df_h['Переменная2'] = list(zip(df_h['xПеременная2'],df_h['yПеременная2']))

df_h = df_h.drop(columns =['xОпорная1', 'xОпорная2', 'yОпорная1','yОпорная2',
                           'xПеременная1','xПеременная2', 'yПеременная2','yПеременная1'] )

print(df_h)
df_h.to_csv('h_filter_new.csv', index=False)

In [ ]:
df_k = pd.read_csv('/home/vladislav/Education/3 курс/astro_prac/photometry_of_variable_stars/data/k_filter_new.csv')
df_j = pd.read_csv('/home/vladislav/Education/3 курс/astro_prac/photometry_of_variable_stars/data/j_filter_new.csv'))
df_h = pd.read_csv('/home/vladislav/Education/3 курс/astro_prac/photometry_of_variable_stars/data/h_filter_new.csv'))

our_magnit = []
our_times = []

